# Optimizer Comparison Analysis

This notebook analyzes and compares the performance of different optimizers for TQFM training on the Digits dataset.

**Optimizers compared:**
- COBYLA (Constrained Optimization BY Linear Approximation)
- SPSA (Simultaneous Perturbation Stochastic Approximation)
- ADAM (Adaptive Moment Estimation)
- L_BFGS_B (Limited-memory BFGS with bounds)
- SLSQP (Sequential Least SQuares Programming)

**Analysis includes:**
1. Loss convergence curves
2. Final loss values comparison
3. Training time comparison
4. Statistical analysis across multiple runs

## 1. Import Required Libraries

In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd())))

import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
from scipy import stats
from src.utils import class_similarity, calculate_accuracy

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 2. Load Results from HPC

In [ ]:
# Configuration
results_dir = Path("../results_optimizer")
depth = 1
ansatz = "TwoLocal"
maxiter = 100000
num_runs = 10

# Optimizers to compare (only 3 optimizers)
optimizers = ["COBYLA", "SPSA", "ADAM"]

print(f"Results directory: {results_dir}")
print(f"Configuration: depth={depth}, ansatz={ansatz}, maxiter={maxiter}")
print(f"Number of runs per optimizer: {num_runs}")
print(f"Optimizers: {optimizers}")

# Load data labels for accuracy calculation
print(f"\nLoading dataset...")
loaded_data = np.load('../data/digits_8features_data.npz')
X_train = loaded_data['X_train']
y_train = loaded_data['y_train']
X_val = loaded_data['X_val']
y_val = loaded_data['y_val']
X_test = loaded_data['X_test']
y_test = loaded_data['y_test']

print(f"Training X shape: {X_train.shape}, y shape: {y_train.shape}")
print(f"Validation X shape: {X_val.shape}, y shape: {y_val.shape}")
print(f"Testing X shape: {X_test.shape}, y shape: {y_test.shape}")

In [ ]:
# Load all results (models and kernels)
optimizer_results = {}

for optimizer in optimizers:
    optimizer_results[optimizer] = {
        'loss_histories': [],
        'final_losses': [],
        'optimal_values': [],
        'num_iterations': [],
        'run_ids': [],
        'kernels_train': [],
        'kernels_val': [],
        'kernels_test': [],
        'separability_ratios': [],
        'val_accuracies': [],
        'test_accuracies': [],
        'best_C_values': []
    }
    
    for run_id in range(1, num_runs + 1):
        # Load model
        model_filename = f"tqfm_digits_depth{depth}_ansatz{ansatz}_optimizer{optimizer}_iter{maxiter}_run{run_id}.pkl"
        model_filepath = results_dir / model_filename
        
        # Load kernels
        kernel_filename = f"kernels_digits_depth{depth}_ansatz{ansatz}_optimizer{optimizer}_iter{maxiter}_run{run_id}.npz"
        kernel_filepath = results_dir / kernel_filename
        
        if model_filepath.exists() and kernel_filepath.exists():
            # Load model
            with open(model_filepath, 'rb') as f:
                tqfm = pickle.load(f)
            
            # Load kernels
            kernels = np.load(kernel_filepath)
            K_train = kernels['kernel_after']
            K_val = kernels['kernel_val']
            K_test = kernels['kernel_test']
            
            # Calculate separability ratio
            _, _, _, sep_ratio = class_similarity(K_train, y_train)
            
            # Calculate accuracies with optimal C selection
            val_acc, test_acc, best_C = calculate_accuracy(
                K_train, K_val, K_test, y_train, y_val, y_test
            )
            
            # Store results
            optimizer_results[optimizer]['loss_histories'].append(tqfm.loss_history)
            optimizer_results[optimizer]['final_losses'].append(tqfm.loss_history[-1])
            optimizer_results[optimizer]['optimal_values'].append(tqfm.optimal_value)
            optimizer_results[optimizer]['num_iterations'].append(len(tqfm.loss_history))
            optimizer_results[optimizer]['run_ids'].append(run_id)
            optimizer_results[optimizer]['kernels_train'].append(K_train)
            optimizer_results[optimizer]['kernels_val'].append(K_val)
            optimizer_results[optimizer]['kernels_test'].append(K_test)
            optimizer_results[optimizer]['separability_ratios'].append(sep_ratio)
            optimizer_results[optimizer]['val_accuracies'].append(val_acc)
            optimizer_results[optimizer]['test_accuracies'].append(test_acc)
            optimizer_results[optimizer]['best_C_values'].append(best_C)
        else:
            if not model_filepath.exists():
                print(f"Warning: Model file not found - {model_filename}")
            if not kernel_filepath.exists():
                print(f"Warning: Kernel file not found - {kernel_filename}")

# Print summary
print("\n" + "="*70)
print("Loaded Results Summary:")
print("="*70)
for optimizer in optimizers:
    num_loaded = len(optimizer_results[optimizer]['loss_histories'])
    print(f"{optimizer:12s}: {num_loaded}/{num_runs} runs loaded")
print("="*70)

## 3. Select Best Run for Each Optimizer

For each optimizer, select the run with the lowest final loss (best performance).

## 4. Accuracy Analysis - All Runs

## 6. Compare Best Runs - Loss Convergence

In [ ]:
# Plot loss convergence curves for best runs only
fig, ax = plt.subplots(figsize=(14, 8))

optimizer_colors = {
    'COBYLA': 'blue',
    'SPSA': 'green',
    'ADAM': 'red'
}

for optimizer in optimizers:
    if optimizer in best_runs:
        loss_hist = best_runs[optimizer]['loss_history']
        run_id = best_runs[optimizer]['run_id']
        
        ax.plot(loss_hist, linewidth=2.5, 
               label=f'{optimizer} (Run {run_id})',
               color=optimizer_colors[optimizer])

ax.set_xlabel('Iteration', fontsize=14)
ax.set_ylabel('Loss E(θ)', fontsize=14)
ax.set_title('Optimizer Comparison: Best Run Loss Convergence', fontsize=16, fontweight='bold')
ax.legend(fontsize=12, loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_ylim(bottom=0)
plt.tight_layout()
plt.savefig('../results_img/optimizer_best_runs_convergence.png', dpi=300, bbox_inches='tight')
plt.show()

print("Figure saved: results_img/optimizer_best_runs_convergence.png")

## 7. All Runs Visualization (with Best Highlighted)

In [ ]:
# Plot all runs with best run highlighted for each optimizer
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, optimizer in enumerate(optimizers):
    ax = axes[idx]
    
    # Plot all runs in light color
    for run_idx, loss_hist in enumerate(optimizer_results[optimizer]['loss_histories']):
        run_id = optimizer_results[optimizer]['run_ids'][run_idx]
        is_best = (run_id == best_runs[optimizer]['run_id'])
        
        if is_best:
            # Plot best run with thick line
            ax.plot(loss_hist, linewidth=3, 
                   color=optimizer_colors[optimizer], 
                   label=f'Best (Run {run_id})', zorder=10)
        else:
            # Plot other runs with thin, transparent lines
            ax.plot(loss_hist, linewidth=1, alpha=0.3, 
                   color=optimizer_colors[optimizer])
    
    ax.set_xlabel('Iteration', fontsize=12)
    ax.set_ylabel('Loss E(θ)', fontsize=12)
    ax.set_title(f'{optimizer}', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=10)
    ax.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig('../results_img/optimizer_all_runs_with_best.png', dpi=300, bbox_inches='tight')
plt.show()

print("Figure saved: results_img/optimizer_all_runs_with_best.png")

## 8. Final Loss Comparison - All Runs

In [ ]:
# Box plot and bar plot showing all runs with best highlighted
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Box plot - All runs
data_for_boxplot = []
labels_for_boxplot = []
for optimizer in optimizers:
    if len(optimizer_results[optimizer]['final_losses']) > 0:
        data_for_boxplot.append(optimizer_results[optimizer]['final_losses'])
        labels_for_boxplot.append(optimizer)

bp = ax1.boxplot(data_for_boxplot, labels=labels_for_boxplot, patch_artist=True)
for patch, optimizer in zip(bp['boxes'], labels_for_boxplot):
    patch.set_facecolor(optimizer_colors[optimizer])
    patch.set_alpha(0.6)

# Add best run markers
for idx, optimizer in enumerate(labels_for_boxplot):
    best_loss = best_runs[optimizer]['final_loss']
    ax1.scatter(idx + 1, best_loss, color='red', s=200, marker='*', 
               zorder=10, edgecolors='black', linewidths=1.5)

ax1.set_ylabel('Final Loss E(θ)', fontsize=12)
ax1.set_xlabel('Optimizer', fontsize=12)
ax1.set_title('Final Loss Distribution (★ = Best Run)', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Bar plot - Compare best runs only
best_losses = [best_runs[opt]['final_loss'] for opt in optimizers if opt in best_runs]
best_labels = [opt for opt in optimizers if opt in best_runs]
x_pos = np.arange(len(best_labels))

bars = ax2.bar(x_pos, best_losses, alpha=0.8,
               color=[optimizer_colors[opt] for opt in best_labels])

# Add value labels on bars
for i, (bar, loss) in enumerate(zip(bars, best_losses)):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002,
            f'{loss:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

ax2.set_ylabel('Final Loss E(θ)', fontsize=12)
ax2.set_xlabel('Optimizer', fontsize=12)
ax2.set_title('Best Run Final Loss Comparison', fontsize=14, fontweight='bold')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(best_labels)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../results_img/optimizer_final_loss_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Figure saved: results_img/optimizer_final_loss_comparison.png")

## 9. Statistical Summary

In [ ]:
# Statistical summary with all metrics
print("\n" + "="*130)
print("STATISTICAL SUMMARY - ALL RUNS")
print("="*130)

stats_all_runs = []
for optimizer in optimizers:
    if len(optimizer_results[optimizer]['final_losses']) > 0:
        stats_all_runs.append({
            'Optimizer': optimizer,
            'N': len(optimizer_results[optimizer]['final_losses']),
            'Loss': f"{np.mean(optimizer_results[optimizer]['final_losses']):.4f}±{np.std(optimizer_results[optimizer]['final_losses']):.4f}",
            'Sep Ratio': f"{np.mean(optimizer_results[optimizer]['separability_ratios']):.4f}±{np.std(optimizer_results[optimizer]['separability_ratios']):.4f}",
            'Val Acc': f"{np.mean(optimizer_results[optimizer]['val_accuracies']):.4f}±{np.std(optimizer_results[optimizer]['val_accuracies']):.4f}",
            'Test Acc': f"{np.mean(optimizer_results[optimizer]['test_accuracies']):.4f}±{np.std(optimizer_results[optimizer]['test_accuracies']):.4f}",
            'Best C': f"{np.mean(optimizer_results[optimizer]['best_C_values']):.2f}±{np.std(optimizer_results[optimizer]['best_C_values']):.2f}"
        })

df_all_runs = pd.DataFrame(stats_all_runs)
print(df_all_runs.to_string(index=False))

print("\n" + "="*130)
print("BEST RUN COMPARISON")
print("="*130)

stats_best_runs = []
for optimizer in optimizers:
    if optimizer in best_runs:
        stats_best_runs.append({
            'Optimizer': optimizer,
            'Run ID': best_runs[optimizer]['run_id'],
            'Loss': best_runs[optimizer]['final_loss'],
            'Sep Ratio': best_runs[optimizer]['separability_ratio'],
            'Val Acc': best_runs[optimizer]['val_accuracy'],
            'Test Acc': best_runs[optimizer]['test_accuracy'],
            'Best C': best_runs[optimizer]['best_C'],
            'Iterations': best_runs[optimizer]['num_iterations']
        })

df_best_runs = pd.DataFrame(stats_best_runs)
df_best_runs = df_best_runs.sort_values('Loss')
print(df_best_runs.to_string(index=False))
print("="*130)

## 10. Convergence Speed Analysis - Best Runs

In [ ]:
# Analyze convergence speed for best runs
thresholds = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

convergence_table = []

print("\n" + "="*70)
print("CONVERGENCE SPEED - Iterations to reach loss threshold")
print("="*70)

for optimizer in optimizers:
    if optimizer in best_runs:
        loss_hist = np.array(best_runs[optimizer]['loss_history'])
        row = {'Optimizer': optimizer}
        
        for threshold in thresholds:
            below_threshold = np.where(loss_hist < threshold)[0]
            if len(below_threshold) > 0:
                iterations = below_threshold[0]
                row[f'{threshold}'] = iterations
            else:
                row[f'{threshold}'] = np.nan
        
        convergence_table.append(row)

df_convergence = pd.DataFrame(convergence_table)
print(df_convergence.to_string(index=False))
print("="*70)

# Plot convergence speed
fig, ax = plt.subplots(figsize=(14, 7))

for optimizer in optimizers:
    if optimizer in best_runs:
        iterations_list = []
        for threshold in thresholds:
            loss_hist = np.array(best_runs[optimizer]['loss_history'])
            below_threshold = np.where(loss_hist < threshold)[0]
            if len(below_threshold) > 0:
                iterations_list.append(below_threshold[0])
            else:
                iterations_list.append(np.nan)
        
        ax.plot(thresholds, iterations_list, marker='o', linewidth=2.5, markersize=8,
               label=f"{optimizer} (Run {best_runs[optimizer]['run_id']})",
               color=optimizer_colors[optimizer])

ax.set_xlabel('Loss Threshold', fontsize=14)
ax.set_ylabel('Iterations to Reach Threshold', fontsize=14)
ax.set_title('Convergence Speed Comparison - Best Runs', fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
ax.invert_xaxis()
plt.tight_layout()
plt.savefig('../results_img/optimizer_convergence_speed.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nFigure saved: results_img/optimizer_convergence_speed.png")

## 11. Final Summary and Recommendation

In [ ]:
# Final comprehensive summary
print("\n" + "="*100)
print("FINAL SUMMARY - OPTIMIZER COMPARISON")
print("="*100)

if len(df_best_runs) > 0:
    best_optimizer = df_best_runs.iloc[0]['Optimizer']
    best_run_id = df_best_runs.iloc[0]['Run ID']
    best_loss = df_best_runs.iloc[0]['Loss']
    best_test_acc = df_best_runs.iloc[0]['Test Acc']
    best_sep_ratio = df_best_runs.iloc[0]['Sep Ratio']
    
    print(f"\n🏆 BEST OPTIMIZER (by lowest loss): {best_optimizer}")
    print(f"   Best Run ID: {best_run_id}")
    print(f"   Final Loss: {best_loss:.6f}")
    print(f"   Separability Ratio: {best_sep_ratio:.6f}")
    print(f"   Test Accuracy: {best_test_acc:.4f} ({best_test_acc*100:.2f}%)")
    print(f"   Best C parameter: {df_best_runs.iloc[0]['Best C']:.3f}")
    
    # Find best by test accuracy
    best_acc_idx = df_best_runs['Test Acc'].idxmax()
    best_acc_optimizer = df_best_runs.loc[best_acc_idx, 'Optimizer']
    best_acc_value = df_best_runs.loc[best_acc_idx, 'Test Acc']
    best_acc_run = df_best_runs.loc[best_acc_idx, 'Run ID']
    
    print(f"\n🎯 BEST OPTIMIZER (by test accuracy): {best_acc_optimizer}")
    print(f"   Run ID: {best_acc_run}")
    print(f"   Test Accuracy: {best_acc_value:.4f} ({best_acc_value*100:.2f}%)")
    print(f"   Loss: {df_best_runs.loc[best_acc_idx, 'Loss']:.6f}")
    
    # Find best by separability ratio
    best_sep_idx = df_best_runs['Sep Ratio'].idxmax()
    best_sep_optimizer = df_best_runs.loc[best_sep_idx, 'Optimizer']
    best_sep_value = df_best_runs.loc[best_sep_idx, 'Sep Ratio']
    
    print(f"\n📊 BEST OPTIMIZER (by separability ratio): {best_sep_optimizer}")
    print(f"   Separability Ratio: {best_sep_value:.6f}")
    print(f"   Test Accuracy: {df_best_runs.loc[best_sep_idx, 'Test Acc']:.4f}")
    
    print(f"\n📈 RANKING BY METRICS:")
    print(f"\n   By Lowest Loss:")
    for idx, row in df_best_runs.iterrows():
        print(f"      {idx+1}. {row['Optimizer']:10s} - Loss: {row['Loss']:.6f}, Test Acc: {row['Test Acc']:.4f}")
    
    df_by_acc = df_best_runs.sort_values('Test Acc', ascending=False)
    print(f"\n   By Highest Test Accuracy:")
    for idx, row in df_by_acc.iterrows():
        print(f"      {idx+1}. {row['Optimizer']:10s} - Test Acc: {row['Test Acc']:.4f}, Loss: {row['Loss']:.6f}")
    
    df_by_sep = df_best_runs.sort_values('Sep Ratio', ascending=False)
    print(f"\n   By Highest Separability Ratio:")
    for idx, row in df_by_sep.iterrows():
        print(f"      {idx+1}. {row['Optimizer']:10s} - Sep: {row['Sep Ratio']:.6f}, Test Acc: {row['Test Acc']:.4f}")
    
    print(f"\n? VARIABILITY ANALYSIS (across 10 runs):")
    for optimizer in optimizers:
        if len(optimizer_results[optimizer]['test_accuracies']) > 0:
            print(f"   {optimizer:10s}:")
            
            mean_loss = np.mean(optimizer_results[optimizer]['final_losses'])
            std_loss = np.std(optimizer_results[optimizer]['final_losses'])
            cv_loss = (std_loss/mean_loss*100) if mean_loss != 0 else 0
            print(f"      Loss:      {mean_loss:.6f} ± {std_loss:.6f} (CV={cv_loss:.2f}%)")
            
            mean_sep = np.mean(optimizer_results[optimizer]['separability_ratios'])
            std_sep = np.std(optimizer_results[optimizer]['separability_ratios'])
            cv_sep = (std_sep/mean_sep*100) if mean_sep != 0 else 0
            print(f"      Sep Ratio: {mean_sep:.6f} ± {std_sep:.6f} (CV={cv_sep:.2f}%)")
            
            mean_acc = np.mean(optimizer_results[optimizer]['test_accuracies'])
            std_acc = np.std(optimizer_results[optimizer]['test_accuracies'])
            cv_acc = (std_acc/mean_acc*100) if mean_acc != 0 else 0
            print(f"      Test Acc:  {mean_acc:.4f} ± {std_acc:.4f} (CV={cv_acc:.2f}%)")
    
    print(f"\n💡 KEY FINDINGS:")
    print(f"   1. Best overall performance: {best_optimizer} (Run {best_run_id})")
    print(f"      - Lowest loss: {best_loss:.6f}")
    print(f"      - Test accuracy: {best_test_acc:.4f}")
    print(f"      - Separability ratio: {best_sep_ratio:.6f}")
    print(f"   2. Higher separability ratio generally correlates with better accuracy")
    print(f"   3. Lower loss generally leads to better separability and accuracy")
    print(f"   4. Initial parameters significantly affect results (CV shows variability)")
    print(f"   5. Optimal C parameter varies: {df_best_runs['Best C'].min():.3f} to {df_best_runs['Best C'].max():.3f}")

print("\n" + "="*100)

In [ ]:
# Select best run (lowest final loss) for each optimizer
best_runs = {}

print("\n" + "="*100)
print("BEST RUN SELECTION (Lowest Final Loss)")
print("="*100)

for optimizer in optimizers:
    if len(optimizer_results[optimizer]['final_losses']) > 0:
        # Find index of run with minimum final loss
        best_idx = np.argmin(optimizer_results[optimizer]['final_losses'])
        
        best_runs[optimizer] = {
            'run_id': optimizer_results[optimizer]['run_ids'][best_idx],
            'loss_history': optimizer_results[optimizer]['loss_histories'][best_idx],
            'final_loss': optimizer_results[optimizer]['final_losses'][best_idx],
            'optimal_value': optimizer_results[optimizer]['optimal_values'][best_idx],
            'num_iterations': optimizer_results[optimizer]['num_iterations'][best_idx],
            'kernel_train': optimizer_results[optimizer]['kernels_train'][best_idx],
            'kernel_val': optimizer_results[optimizer]['kernels_val'][best_idx],
            'kernel_test': optimizer_results[optimizer]['kernels_test'][best_idx],
            'separability_ratio': optimizer_results[optimizer]['separability_ratios'][best_idx],
            'val_accuracy': optimizer_results[optimizer]['val_accuracies'][best_idx],
            'test_accuracy': optimizer_results[optimizer]['test_accuracies'][best_idx],
            'best_C': optimizer_results[optimizer]['best_C_values'][best_idx]
        }
        
        print(f"\n{optimizer}:")
        print(f"  Best Run ID: {best_runs[optimizer]['run_id']}")
        print(f"  Final Loss: {best_runs[optimizer]['final_loss']:.6f}")
        print(f"  Separability Ratio: {best_runs[optimizer]['separability_ratio']:.6f}")
        print(f"  Iterations: {best_runs[optimizer]['num_iterations']}")
        print(f"  Best C: {best_runs[optimizer]['best_C']:.3f}")
        print(f"  Val Accuracy: {best_runs[optimizer]['val_accuracy']:.4f}")
        print(f"  Test Accuracy: {best_runs[optimizer]['test_accuracy']:.4f}")
        
        # Show all runs for comparison
        print(f"  All runs (Loss | Sep Ratio | Test Acc):")
        for i, (run_id, loss, sep, test_acc) in enumerate(zip(
                optimizer_results[optimizer]['run_ids'], 
                optimizer_results[optimizer]['final_losses'],
                optimizer_results[optimizer]['separability_ratios'],
                optimizer_results[optimizer]['test_accuracies'])):
            marker = " ← BEST" if i == best_idx else ""
            print(f"    Run {run_id:2d}: {loss:.6f} | {sep:.6f} | {test_acc:.4f}{marker}")

print("\n" + "="*100)

In [ ]:
# Accuracy and Separability comparison across all runs
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

optimizer_colors = {
    'COBYLA': 'blue',
    'SPSA': 'green',
    'ADAM': 'red'
}

# 1. Validation Accuracy
ax = axes[0, 0]
data_box = [optimizer_results[opt]['val_accuracies'] for opt in optimizers if len(optimizer_results[opt]['val_accuracies']) > 0]
labels_box = [opt for opt in optimizers if len(optimizer_results[opt]['val_accuracies']) > 0]
bp = ax.boxplot(data_box, labels=labels_box, patch_artist=True)
for patch, optimizer in zip(bp['boxes'], labels_box):
    patch.set_facecolor(optimizer_colors[optimizer])
    patch.set_alpha(0.6)
for i, optimizer in enumerate(labels_box):
    best_acc = best_runs[optimizer]['val_accuracy']
    ax.scatter(i + 1, best_acc, color='red', s=200, marker='*', zorder=10, edgecolors='black', linewidths=1.5)
ax.set_ylabel('Validation Accuracy', fontsize=12)
ax.set_title('Validation Accuracy (★ = Best Run)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# 2. Test Accuracy
ax = axes[0, 1]
data_box = [optimizer_results[opt]['test_accuracies'] for opt in optimizers if len(optimizer_results[opt]['test_accuracies']) > 0]
labels_box = [opt for opt in optimizers if len(optimizer_results[opt]['test_accuracies']) > 0]
bp = ax.boxplot(data_box, labels=labels_box, patch_artist=True)
for patch, optimizer in zip(bp['boxes'], labels_box):
    patch.set_facecolor(optimizer_colors[optimizer])
    patch.set_alpha(0.6)
for i, optimizer in enumerate(labels_box):
    best_acc = best_runs[optimizer]['test_accuracy']
    ax.scatter(i + 1, best_acc, color='red', s=200, marker='*', zorder=10, edgecolors='black', linewidths=1.5)
ax.set_ylabel('Test Accuracy', fontsize=12)
ax.set_title('Test Accuracy (★ = Best Run)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# 3. Separability Ratio
ax = axes[1, 0]
data_box = [optimizer_results[opt]['separability_ratios'] for opt in optimizers if len(optimizer_results[opt]['separability_ratios']) > 0]
labels_box = [opt for opt in optimizers if len(optimizer_results[opt]['separability_ratios']) > 0]
bp = ax.boxplot(data_box, labels=labels_box, patch_artist=True)
for patch, optimizer in zip(bp['boxes'], labels_box):
    patch.set_facecolor(optimizer_colors[optimizer])
    patch.set_alpha(0.6)
for i, optimizer in enumerate(labels_box):
    best_sep = best_runs[optimizer]['separability_ratio']
    ax.scatter(i + 1, best_sep, color='red', s=200, marker='*', zorder=10, edgecolors='black', linewidths=1.5)
ax.set_ylabel('Separability Ratio', fontsize=12)
ax.set_title('Separability Ratio (★ = Best Run)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# 4. Best C Parameter
ax = axes[1, 1]
data_box = [optimizer_results[opt]['best_C_values'] for opt in optimizers if len(optimizer_results[opt]['best_C_values']) > 0]
labels_box = [opt for opt in optimizers if len(optimizer_results[opt]['best_C_values']) > 0]
bp = ax.boxplot(data_box, labels=labels_box, patch_artist=True)
for patch, optimizer in zip(bp['boxes'], labels_box):
    patch.set_facecolor(optimizer_colors[optimizer])
    patch.set_alpha(0.6)
for i, optimizer in enumerate(labels_box):
    best_c = best_runs[optimizer]['best_C']
    ax.scatter(i + 1, best_c, color='red', s=200, marker='*', zorder=10, edgecolors='black', linewidths=1.5)
ax.set_ylabel('Best C Parameter', fontsize=12)
ax.set_title('Optimal C Parameter (★ = Best Run)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../results_img/optimizer_accuracy_separability_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Figure saved: results_img/optimizer_accuracy_separability_comparison.png")

## 5. Loss vs Accuracy Correlation

In [ ]:
# Correlation analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Loss vs Test Accuracy
ax = axes[0, 0]
for optimizer in optimizers:
    final_losses = optimizer_results[optimizer]['final_losses']
    test_accs = optimizer_results[optimizer]['test_accuracies']
    
    if len(final_losses) > 0:
        ax.scatter(final_losses, test_accs, s=100, alpha=0.6,
                   label=optimizer, color=optimizer_colors[optimizer])
        
        # Highlight best run
        best_loss = best_runs[optimizer]['final_loss']
        best_acc = best_runs[optimizer]['test_accuracy']
        ax.scatter(best_loss, best_acc, s=300, marker='*', 
                   color=optimizer_colors[optimizer], edgecolors='black', 
                   linewidths=2, zorder=10)

ax.set_xlabel('Final Loss E(θ)', fontsize=12)
ax.set_ylabel('Test Accuracy', fontsize=12)
ax.set_title('Loss vs Test Accuracy (★ = Best Run)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Plot 2: Separability Ratio vs Test Accuracy
ax = axes[0, 1]
for optimizer in optimizers:
    sep_ratios = optimizer_results[optimizer]['separability_ratios']
    test_accs = optimizer_results[optimizer]['test_accuracies']
    
    if len(sep_ratios) > 0:
        ax.scatter(sep_ratios, test_accs, s=100, alpha=0.6,
                   label=optimizer, color=optimizer_colors[optimizer])
        
        # Highlight best run
        best_sep = best_runs[optimizer]['separability_ratio']
        best_acc = best_runs[optimizer]['test_accuracy']
        ax.scatter(best_sep, best_acc, s=300, marker='*', 
                   color=optimizer_colors[optimizer], edgecolors='black', 
                   linewidths=2, zorder=10)

ax.set_xlabel('Separability Ratio', fontsize=12)
ax.set_ylabel('Test Accuracy', fontsize=12)
ax.set_title('Separability Ratio vs Test Accuracy (★ = Best Run)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Plot 3: Loss vs Separability Ratio
ax = axes[1, 0]
for optimizer in optimizers:
    final_losses = optimizer_results[optimizer]['final_losses']
    sep_ratios = optimizer_results[optimizer]['separability_ratios']
    
    if len(final_losses) > 0:
        ax.scatter(final_losses, sep_ratios, s=100, alpha=0.6,
                   label=optimizer, color=optimizer_colors[optimizer])
        
        # Highlight best run
        best_loss = best_runs[optimizer]['final_loss']
        best_sep = best_runs[optimizer]['separability_ratio']
        ax.scatter(best_loss, best_sep, s=300, marker='*', 
                   color=optimizer_colors[optimizer], edgecolors='black', 
                   linewidths=2, zorder=10)

ax.set_xlabel('Final Loss E(θ)', fontsize=12)
ax.set_ylabel('Separability Ratio', fontsize=12)
ax.set_title('Loss vs Separability Ratio (★ = Best Run)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Plot 4: Val Accuracy vs Test Accuracy
ax = axes[1, 1]
for optimizer in optimizers:
    val_accs = optimizer_results[optimizer]['val_accuracies']
    test_accs = optimizer_results[optimizer]['test_accuracies']
    
    if len(val_accs) > 0:
        ax.scatter(val_accs, test_accs, s=100, alpha=0.6,
                   label=optimizer, color=optimizer_colors[optimizer])
        
        # Highlight best run
        best_val_acc = best_runs[optimizer]['val_accuracy']
        best_test_acc = best_runs[optimizer]['test_accuracy']
        ax.scatter(best_val_acc, best_test_acc, s=300, marker='*', 
                   color=optimizer_colors[optimizer], edgecolors='black', 
                   linewidths=2, zorder=10)

# Add diagonal line
ax.plot([0, 1], [0, 1], 'k--', alpha=0.3, linewidth=1)

ax.set_xlabel('Validation Accuracy', fontsize=12)
ax.set_ylabel('Test Accuracy', fontsize=12)
ax.set_title('Validation vs Test Accuracy (★ = Best Run)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim([0.5, 1])
ax.set_ylim([0.5, 1])

plt.tight_layout()
plt.savefig('../results_img/optimizer_correlation_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("Figure saved: results_img/optimizer_correlation_analysis.png")